# Music Genre Classification

Classify the musical genre of a given audio track - from data to deployed model. Adapted from the work of Huang, Serafini, and Pugh [1].

## Authors
- Sebastian Lehrig <sebastian.lehrig1@ibm.com>
- Marvin Giessing <MARVING@de.ibm.com>

## License
Apache-2.0 License

## References
[1] original paper: http://cs229.stanford.edu/proj2018/report/21.pdf

[2] code: https://github.com/derekahuang/Music-Classification

[3] Preprocessed Data: https://drive.google.com/file/d/12mCgkvbmissLh2Vop0bp_t98G8QCaV1E/view?usp=sharing

## 0.) Imports & Constants

In [ ]:
import IPython.display
import json
import kfp
from kfp.components import InputPath, OutputPath
import kfp.dsl as dsl
from kfp.dsl import PipelineConf, data_passing_methods
from kubernetes.client.models import V1Volume, V1PersistentVolumeClaimVolumeSource
import librosa as lb
from librosa import display
import math
import numpy as np
import os
import pylab
import requests
from typing import List, NamedTuple

%load_ext lab_black

In [ ]:
# Environment-specific configurations
#
# %env CLUSTER_CONFIGURATION_SECRET remote-power-cluster
# %env CLUSTER_CONFIGURATION_SECRET remote-x86-cluster
# %env CLUSTER_CONFIGURATION_SECRET remote-x86-telekom-cluster
# %env TRAINING_GPUS 1
# %env TRAINING_NODE_SELECTOR nvidia.com/gpu.product: "Tesla-V100-SXM2-32GB"
# %env TRAINING_NODE_SELECTOR kubernetes.io/hostname: node2
#
# Reset:
# del os.environ['CLUSTER_CONFIGURATION_SECRET']
# del os.environ['TRAINING_GPUS']
# del os.environ['TRAINING_NODE_SELECTOR']

In [ ]:
COMPONENT_CATALOG_FOLDER = f"{os.getenv('HOME')}/components"
COMPONENT_CATALOG_GIT = "https://github.com/lehrig/kubeflow-ppc64le-components.git"
COMPONENT_CATALOG_RELEASE = "main"

LOAD_DATASET_COMPONENT = (
    f"{COMPONENT_CATALOG_FOLDER}/data-collection/load-dataset/component.yaml"
)
TRAIN_MODEL_COMPONENT = (
    f"{COMPONENT_CATALOG_FOLDER}/model-building/train-model-job/component.yaml"
)
PLOT_CONFUSION_MATRIX_COMPONENT = (
    f"{COMPONENT_CATALOG_FOLDER}/model-building/plot-confusion-matrix/component.yaml"
)
CONVERT_MODEL_TO_ONNX_COMPONENT = (
    f"{COMPONENT_CATALOG_FOLDER}/model-building/convert-to-onnx/component.yaml"
)
UPLOAD_MODEL_COMPONENT = (
    f"{COMPONENT_CATALOG_FOLDER}/model-building/upload-model/component.yaml"
)
DEPLOY_MODEL_WITH_KSERVE_COMPONENT = f"{COMPONENT_CATALOG_FOLDER}/model-deployment/deploy-model-with-kserve/component.yaml"

BASE_IMAGE = "quay.io/ibm/kubeflow-notebook-image-ppc64le:latest"

ARGUMENTS = {
    "blackboard": "artefacts",
    "dataset_url": "Lehrig/GTZAN-Collection",
    "dataset_configuration": "mel_spectrograms",
    "dataset_label_columns": ["genre"],
    "model_name": "music-classification",
    "cluster_configuration_secret": os.getenv(
        "CLUSTER_CONFIGURATION_SECRET", default=""
    ),
    "training_gpus": os.getenv("TRAINING_GPUS", default="1"),
    "training_node_selector": os.getenv("TRAINING_NODE_SELECTOR", default=""),
}
MODEL_NAME = ARGUMENTS["model_name"]

with open("/var/run/secrets/kubernetes.io/serviceaccount/namespace") as f:
    NAMESPACE = f.read()

ARGUMENTS

## 1.) Let's start with creating a client object for interaction

In [ ]:
client = kfp.Client()

## 2.) The main part consists of defining the end-to-end workflow functions and create components from them
### 2.0) Load component catalog

In [ ]:
!git clone --branch $COMPONENT_CATALOG_RELEASE $COMPONENT_CATALOG_GIT $COMPONENT_CATALOG_FOLDER

### 2.1) Load dataset (by reusing a Kubeflow component)

In [ ]:
load_dataset_comp = kfp.components.load_component_from_file(LOAD_DATASET_COMPONENT)

### 2.2) Preprocess data (one hot encoding etc.)

In [ ]:
def preprocess_dataset(
    dataset_dir: InputPath(str),
    train_dataset_dir: OutputPath(str),
    validation_dataset_dir: OutputPath(str),
    test_dataset_dir: OutputPath(str),
    batch_size: int = 200,
):
    """Split data into train/dev/test data. Saves result into `prep_dataset_dir`."""

    from datasets import load_from_disk
    import numpy as np
    import os
    from transformers import DefaultDataCollator

    print(f"Loading input dataset from {dataset_dir}...")
    dataset = load_from_disk(dataset_dir)

    # Preprocess
    num_classes = dataset["train"].features["genre"].num_classes
    one_hot_matrix = np.eye(num_classes)

    def process(examples):
        examples["genre"] = [one_hot_matrix[genre] for genre in examples["genre"]]
        return examples

    prep_dataset = dataset.map(
        process, batched=True, batch_size=batch_size, num_proc=2, keep_in_memory=True
    )

    def save_as_tfdataset(
        dataset, columns, label_columns, data_collator, directory, shuffle
    ):
        import tensorflow as tf

        tf_dataset = dataset.to_tf_dataset(
            columns=columns,
            label_cols=label_columns,
            shuffle=shuffle,
            batch_size=batch_size,
            collate_fn=data_collator,
        )

        print(f"Saving pre-processed dataset to '{directory}'...")
        if not os.path.exists(directory):
            os.makedirs(directory)
        tf.data.experimental.save(tf_dataset, directory)

        print(f"Pre-processed dataset saved. Contents of '{directory}':")
        print(os.listdir(directory))

    # prep_dataset = prep_dataset.with_format("numpy")
    data_collator = DefaultDataCollator(return_tensors="tf")
    columns = ["mel_spectrogram"]
    label_columns = ["genre"]
    save_as_tfdataset(
        prep_dataset["train"],
        columns,
        label_columns,
        data_collator,
        train_dataset_dir,
        True,
    )
    save_as_tfdataset(
        prep_dataset["validation"],
        columns,
        label_columns,
        data_collator,
        validation_dataset_dir,
        False,
    )
    save_as_tfdataset(
        prep_dataset["test"],
        columns,
        label_columns,
        data_collator,
        test_dataset_dir,
        False,
    )

    print("Finished.")


preprocess_dataset_comp = kfp.components.create_component_from_func(
    func=preprocess_dataset, base_image=BASE_IMAGE
)

### 2.3) Train the model

In [ ]:
def train_model(
    train_dataset_dir: InputPath(str),
    validation_dataset_dir: InputPath(str),
    model_dir: OutputPath(str),
    epochs: int = 100,
    batch_size: int = 200,
):
    """Trains CNN model. Once trained, the model is persisted to `model_dir`."""

    import os
    import tensorflow as tf
    from tensorflow.keras.callbacks import (
        EarlyStopping,
        ModelCheckpoint,
        ReduceLROnPlateau,
        TensorBoard,
    )
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import (
        BatchNormalization,
        Conv2D,
        Dense,
        Dropout,
        Flatten,
        MaxPooling2D,
    )
    from tensorflow.keras import regularizers
    import time

    def load_datasets():
        train_dataset = tf.data.experimental.load(train_dataset_dir)
        validation_dataset = tf.data.experimental.load(validation_dataset_dir)
        return (train_dataset, validation_dataset)

    def build_model():
        model = Sequential()

        # Feature Learning Layers
        model.add(
            Conv2D(
                64,
                kernel_size=(4, 4),
                activation="relu",
                kernel_regularizer=regularizers.l2(0.04),
                input_shape=(64, 173, 1),
            )
        )
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 4)))

        model.add(
            Conv2D(
                64, (3, 5), activation="relu", kernel_regularizer=regularizers.l2(0.04)
            )
        )
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))

        model.add(
            Conv2D(
                64, (2, 2), activation="relu", kernel_regularizer=regularizers.l2(0.04)
            )
        )
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))

        # Classification Layers
        model.add(Flatten())
        model.add(
            Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.04))
        )
        model.add(Dropout(0.5))
        model.add(
            Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.04))
        )
        model.add(Dense(10, activation="softmax"))

        return model

    print("Loading datasets...")
    train_dataset, validation_dataset = load_datasets()

    print("Building model...")
    model = build_model()
    print(model.summary())

    print("Compiling model...")
    model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["categorical_accuracy"],
    )

    print("Initiallizing training callbacks...")
    callbacks = [
        EarlyStopping(monitor="val_loss", patience=20, verbose=0, mode="min"),
        ModelCheckpoint(
            f"{model_dir}/best_model.keras",
            monitor="val_loss",
            save_best_only=True,
            mode="min",
        ),
        ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.1,
            patience=7,
            verbose=1,
            min_delta=0.0001,
            mode="min",
        ),
        TensorBoard(log_dir="./logs", histogram_freq=1),
    ]
    # TODO
    # Connect TensorBoard to s3://minio-service-kubeflow.apps.b2s001.pbm.ihost.com/mlpipeline/tensorboard
    # Will need tensorflow-io[tensorflow] package for that
    # See: https://github.com/tensorflow/tensorflow/issues/40302#issuecomment-918917472

    print("Starting model training...")
    start = time.time()
    hist = model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=epochs,
        callbacks=callbacks,
    )
    print("\n\nTraining took ", time.time() - start, "seconds")

    print("Model train history:")
    print(hist.history)

    print(f"Saving model to: {model_dir}")
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model.save(model_dir)
    print(f"Model saved to: {model_dir}")

    print("Finished.")


train_specification = kfp.components.func_to_component_text(func=train_model)

In [ ]:
train_model_comp = kfp.components.load_component_from_file(TRAIN_MODEL_COMPONENT)

### 2.4) Evaluate model with validation data

In [ ]:
def evaluate_model(
    test_dataset_dir: InputPath(str), model_dir: InputPath(str), batch_size: int = 20
) -> NamedTuple("EvaluationOutput", [("mlpipeline_metrics", "Metrics")]):
    """Loads a saved model from file and uses a pre-downloaded dataset for evaluation.
    Model metrics are persisted to `{metrics_path}` for Kubeflow Pipelines metadata."""

    from collections import namedtuple
    import json
    import tensorflow as tf

    test_dataset = tf.data.experimental.load(test_dataset_dir)
    model = tf.keras.models.load_model(model_dir)
    (loss, accuracy) = model.evaluate(test_dataset)

    print((loss, accuracy))

    metrics = {
        "metrics": [
            {"name": "loss", "numberValue": str(loss), "format": "PERCENTAGE"},
            {"name": "accuracy", "numberValue": str(accuracy), "format": "PERCENTAGE"},
        ]
    }

    out_tuple = namedtuple("EvaluationOutput", ["mlpipeline_metrics"])

    return out_tuple(json.dumps(metrics))


evaluate_model_comp = kfp.components.create_component_from_func(
    func=evaluate_model, base_image=BASE_IMAGE
)

### 2.5) Create confusion matrix (by reusing a Kubeflow component)

In [ ]:
plot_confusion_matrix_comp = kfp.components.load_component_from_file(
    PLOT_CONFUSION_MATRIX_COMPONENT
)

### 2.6) Convert model to ONNX (by reusing a Kubeflow component)

In [ ]:
convert_model_to_onnx_comp = kfp.components.load_component_from_file(
    CONVERT_MODEL_TO_ONNX_COMPONENT
)

### 2.7) Upload model to MinIO artifact store (by reusing a Kubeflow component)

In [ ]:
upload_model_comp = kfp.components.load_component_from_file(UPLOAD_MODEL_COMPONENT)

### 2.8) Deploy the model using KServe (by reusing a Kubeflow component)

In [ ]:
deploy_model_with_kserve_comp = kfp.components.load_component_from_file(
    DEPLOY_MODEL_WITH_KSERVE_COMPONENT
)

## 3.) Create the actual pipeline by combining the components

In [ ]:
@dsl.pipeline(
    name="Music Genre Classification",
    description="An example pipeline that performs a music genre classification on audio data",
)
def music_genre_classification_pipeline(
    blackboard: str,
    dataset_url: str,
    dataset_configuration: str,
    dataset_label_columns: List[str],
    model_name: str,
    cluster_configuration_secret: str,
    training_gpus: int,
    training_node_selector: str,
):
    create_blackboard = dsl.VolumeOp(
        name="Create Artefacts Blackboard",
        resource_name=blackboard,
        modes=dsl.VOLUME_MODE_RWO,
        size="4Gi",
        set_owner_reference=True,
    )

    load_dataset_task = load_dataset_comp(
        path=dataset_url,
        configuration=dataset_configuration,
        label_columns=dataset_label_columns,
    )
    load_dataset_task.after(create_blackboard)

    preprocess_dataset_task = preprocess_dataset_comp(
        dataset_dir=load_dataset_task.outputs["dataset_dir"],
    )

    # InputPath and OutputPath like "prep_dataset_dir" & "model_dir":
    # Use name of parameters of train component on right-hand side.
    train_parameters = {
        "train_dataset_dir": "train_dataset_dir",
        "validation_dataset_dir": "validation_dataset_dir",
        "model_dir": "model_dir",
    }

    train_model_task = train_model_comp(
        preprocess_dataset_task.outputs["train_dataset_dir"],
        preprocess_dataset_task.outputs["validation_dataset_dir"],
        train_specification,
        train_parameters,
        model_name=model_name,
        gpus=training_gpus,
        node_selector=training_node_selector,
        cluster_configuration_secret=cluster_configuration_secret,
    )

    evaluate_model_comp(
        preprocess_dataset_task.outputs["test_dataset_dir"],
        train_model_task.outputs["model_dir"],
    )

    plot_confusion_matrix_comp(
        input_columns=["mel_spectrogram"],
        label_columns=load_dataset_task.outputs["labels"],
        test_dataset_dir=preprocess_dataset_task.outputs["test_dataset_dir"],
        model_dir=train_model_task.outputs["model_dir"],
    )

    convert_model_to_onnx_task = convert_model_to_onnx_comp(
        train_model_task.outputs["model_dir"]
    )

    upload_model_task = upload_model_comp(
        convert_model_to_onnx_task.outputs["onnx_model_dir"], model_name=model_name
    )

    deploy_model_with_kserve_task = deploy_model_with_kserve_comp(model_name=model_name)

    deploy_model_with_kserve_task.after(upload_model_task)

## 4.) Run the pipeline within an experiment
Create a pipeline run, using the client you initialized in a prior step.

In [ ]:
# See: https://www.kubeflow.org/docs/components/pipelines/overview/caching/#managing-caching-staleness
def disable_cache_transformer(op):
    if isinstance(op, dsl.ContainerOp):
        op.execution_options.caching_strategy.max_cache_staleness = "P0D"
    else:
        op.add_pod_annotation(
            name="pipelines.kubeflow.org/max_cache_staleness", value="P0D"
        )
    return op


pipeline_conf = PipelineConf()
pipeline_conf.add_op_transformer(disable_cache_transformer)
pipeline_conf.data_passing_method = data_passing_methods.KubernetesVolume(
    volume=V1Volume(
        name=ARGUMENTS["blackboard"],
        persistent_volume_claim=V1PersistentVolumeClaimVolumeSource(
            "{{workflow.name}}-%s" % ARGUMENTS["blackboard"]
        ),
    ),
    path_prefix=f'{ARGUMENTS["blackboard"]}/',
)

client.create_run_from_pipeline_func(
    music_genre_classification_pipeline,
    arguments=ARGUMENTS,
    namespace=NAMESPACE,
    pipeline_conf=pipeline_conf,
)

## 5.) Test model deployment
See API documentation: https://github.com/kserve/kserve/blob/master/docs/predict-api/v2/required_api.md

### 5.1) Check model endpoint availability

In [ ]:
HOST = MODEL_NAME + "-predictor-default." + NAMESPACE
HEADERS = {'Host': HOST}
MODEL_ENDPOINT = "http://" + MODEL_NAME + "-predictor-default/v2/models/" + MODEL_NAME

res = requests.get(MODEL_ENDPOINT, headers=HEADERS)
response = json.loads(res.text)
response

Note you can also do this:
```curl -H "Host: $HOST" $MODEL_ENDPOINT```

### 5.2) Get test audio
See: https://commons.wikimedia.org/wiki/Category:Audio_files_of_blues

In [ ]:
AUDIO = "audio.ogg"

AUDIO_URL = "https://upload.wikimedia.org/wikipedia/commons/7/7c/Boogie_lead_riff.ogg"
# AUDIO_URL = "https://upload.wikimedia.org/wikipedia/commons/9/99/Blues_Rock.ogg"

!wget $AUDIO_URL -O $AUDIO

In [ ]:
IPython.display.Audio(AUDIO)

### 5.3) Convert test audio to 2-seconds Mel Spectrograms

In [ ]:
SR = 22050
N_FFT = 512
HOP_LENGTH = N_FFT // 2
N_MELS = 64


def convert_to_melspecs(filename):
    audios = get_batches(filename)
    return batch_log_melspectrogram(audios)


def get_batches(audio):
    y, sr = lb.load(audio, mono=True)

    duration = lb.core.get_duration(y)

    audios = []
    # prune first 2 seconds and ending (assumption: does not include important data)
    for i in range(2, math.floor(duration), 2):
        y_sample, sr_sample = lb.load(audio, mono=True, offset=i, duration=2.0)
        audios.append(y_sample)

    return audios


def log_melspectrogram(data):
    melspec = lb.feature.melspectrogram(
        y=data, hop_length=HOP_LENGTH, n_fft=N_FFT, n_mels=N_MELS
    )
    return lb.power_to_db(melspec**2)


def batch_log_melspectrogram(data_list):
    melspecs = np.asarray(
        [log_melspectrogram(data_list[i]) for i in range(len(data_list) - 1)]
    )
    melspecs = melspecs.reshape(
        melspecs.shape[0], melspecs.shape[1], melspecs.shape[2], 1
    )
    return melspecs


melspecs = convert_to_melspecs(AUDIO)

### 5.4) Visualize a Mel Spectrogram

In [ ]:
middle = (int)(melspecs.shape[0] / 2)
example_melspec = melspecs[middle]
example_melspec = example_melspec.reshape(
    example_melspec.shape[0], example_melspec.shape[1]
)

pylab.axis("off")  # no axis
pylab.axes(
    [0.0, 0.0, 1.0, 1.0], frameon=False, xticks=[], yticks=[]
)  # Remove the white edge
display.specshow(example_melspec, y_axis="mel", x_axis="time")

### 5.5) Score example Mel Spectrogram against model 

In [ ]:
def score(melspec):
    PREDICT_ENDPOINT = MODEL_ENDPOINT + "/infer"

    payload = {
      "inputs": [{
          "name": "conv2d_input",
          "shape": [1, 64, 173, 1],
          "datatype": "FP32",
          "data": melspec.tolist()
        }
      ]
    }

    res = requests.post(PREDICT_ENDPOINT, headers=HEADERS, data=json.dumps(payload))
    response = json.loads(res.text)
    return response['outputs'][0]['data']


test_score = score(example_melspec)
test_score

In [ ]:
GENRE_LABELS = [
    "Blues",
    "Classical",
    "Country",
    "Disco",
    "Hip hop",
    "Jazz",
    "Metal",
    "Pop",
    "Reggae",
    "Rock"
]

GENRE_LABELS[np.argmax(test_score)]

### 5.6) Score each Mel Spectrogram against deployed model & aggregate results

In [ ]:
probabilities = [0.0 for genre in GENRE_LABELS]
counts = [0 for genre in GENRE_LABELS]

for melspec in melspecs:
    predictions = score(melspec)
    for i in range(len(GENRE_LABELS)):
        probabilities[i] += predictions[i]
    counts[np.argmax(predictions)] += 1

### 5.7) Aggregate results

In [ ]:
for i in range(len(GENRE_LABELS)):
    probabilities[i] = probabilities[i]/len(melspecs)
    print(GENRE_LABELS[i] + ": " + str(probabilities[i]) + " (" + str(counts[i]) + "x)")